In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import pandas as pd
import os
import json
from IPython.display import display

from helpers import renamePostaja, cleanPostaja, lowerSumniki

Step1: DataFrame for each air pollutant
- read each csv
- input into dataframe
- convert into long format

In [ ]:
nevarne_snovi = [("PM10", 1999), ("SO2", 1997), ("NOX", 1997)]
meseci = ["Jan", "Feb", "Mar", "Apr", "Maj","Jun", "Jul", "Avg", "Sep", "Okt", "Nov","Dec"]

In [5]:
pd_dict = {}


for snov, zac_leto in nevarne_snovi:
    dir_path = f"podatki/{snov}"

    for leto, leto_file in enumerate(sorted(os.listdir(dir_path)), start=zac_leto):
        file_path = os.path.join(dir_path, leto_file)
        df = pd.read_csv(file_path, na_values=["", ",", " ", "/"])

        # cleanup
        df = renamePostaja(df)
        df = cleanPostaja(df)

        #add year
        df.insert(1, "Leto", leto)

        #wide to long conversion
        df = df.melt(
            id_vars=["Postaja", "Leto"],
            value_vars=meseci,
            var_name="Mesec",
            value_name=snov,
        )
        if snov not in pd_dict:
            pd_dict[snov] = df
        else:
            pd_dict[snov] = pd.concat([pd_dict[snov], df])

    print(f"Reading complete!")

Reading complete!
Reading complete!
Reading complete!


Step2: Map stations to regions

In [ ]:
map_dict = json.load(open("podatki/slovar.json"))
map_dict = {key: [lowerSumniki(v) for v in values] for key,values in map_dict.items()}

vsi_mappingsi = set([p for postaje in map_dict.values() for p in postaje])
vse_postaje = set([postaja for df in pd_dict.values() for postaja in df['Postaja'].unique()])

vse_postaje - vsi_mappingsi



{'eis krsko',
 'graska gora',
 'ib gregorciceva',
 'ib recica',
 'lj bezigrad',
 'lj biotehnicna fak.',
 'lj biotehniska',
 'lj biotehniska fakulteta',
 'lj celovska',
 'lj vic',
 'ljubljana bez.',
 'ljubljana bezigrad',
 'm. s.-rakican',
 'maribor vrbanski plato',
 'mb center',
 'mb vrbanski pl.',
 'miklavz',
 'mobilna postaja',
 'mobilna tes',
 'ms rakican',
 'murska s. rakican',
 'murska s.-rakican',
 'murska sobota rakican',
 'murska sobota-rakican',
 'ng grcna',
 'pobrezje',
 'rusa',
 'ruse',
 'sostanj',
 'sv.mohor',
 'sveti mohor',
 'topolsica'}

In [ ]:
regije_dict = {}

tmp = pd.Series(map_dict).explode().drop_duplicates()
tmp = pd.Series(tmp.index, tmp)

for snov, leto in nevarne_snovi:
    regije_dict[snov] = pd_dict[snov].copy()
    regije_dict[snov].insert(loc=1, column='Regija', value=regije_dict[snov]['Postaja'].map(tmp))